In [30]:
#13. feladat:

with open('13_feladat.txt', 'r') as f:
    lines = f.read().strip().split('\n')
    #print(lines)
    directories = ['/']
    files_in_dirs = {}
    actual_dir = '/'
    dir_weights = {'/': 0}
    file_weights = {}
    ascendants = {'/': []}
    parents = {'/': []}
    prec = []
    
    for line in lines[1:]:
        items = line.split(' ')
        
        if items[0] == '$' and items[1] == 'cd' and items[2] == '..':
            actual_dir = parents[actual_dir][-1]
        
        elif items[0] == '$' and items[1] == 'cd' and items[2] == '/':
            actual_dir = '/'
            
        elif items[0] == '$' and items[1] == 'cd':
            actual_dir = items[2]
            if items[2] not in list(ascendants.keys()):
                ascendants[items[2]] = []
            
        elif items[0] == '$' and items[1] == 'ls':
            pass    
        
        elif items[0] == 'dir':
            directories.append(items[1])
            ascendants[actual_dir].append(items[1])
            parents[items[1]] = parents[actual_dir] + [actual_dir]
            dir_weights[items[1]] = 0
            
        else:
            dir_weights[actual_dir] += int(items[0])
            for d in parents[actual_dir]:
                dir_weights[d] += int(items[0])
            files_in_dirs[items[1]] = parents[actual_dir] + [actual_dir]
            file_weights[items[1]] = int(items[0])
        
        #print(items)
        
    #print(dir_weights)
    #print(directories)
    value = sum([v for v in list(dir_weights.values()) if v <= 100000])
    print(files_in_dirs)
    print(value)
            
            
          
            
            
        

{'dqqbcfr': ['/'], 'fzjdz': ['/', 'jvtlfbzr', 'gwz', 'ldcmtwvt', 'fzjdz', 'qbpvrr', 'mdvqv'], 'lsvw.lwr': ['/'], 'mjjj.wnq': ['/', 'blrnnv'], 'zzdpcnmt.qbf': ['/', 'sbprmc', 'hfns', 'cvjtpc', 'whqnlc', 'tchbjclg', 'ptfsh', 'rvrsrfl', 'qbpvrr'], 'hnbqlmmg': ['/', 'dqf'], 'lmw.zmd': ['/', 'dqf'], 'tsrwvqbg.tzh': ['/', 'dqf'], 'bddslhrg.twf': ['/', 'dqf', 'mwj'], 'qfccg.crl': ['/', 'wtm'], 'vtzlv': ['/', 'ftj'], 'thwhz.pgp': ['/', 'ftj', 'clchr', 'sbzf'], 'bddslhrg.qzl': ['/', 'ftj', 'hmd'], 'sgddpc': ['/', 'ftj', 'hmd'], 'vjchlsrc.cmh': ['/', 'sbprmc', 'hfns', 'cvjtpc', 'whqnlc', 'tchbjclg', 'ptfsh'], 'bddslhrg': ['/', 'sbprmc', 'hfns', 'cvjtpc', 'whqnlc', 'tchbjclg', 'bljmjwm', 'qlzg', 'ghllcw', 'fzjdz', 'wtvfgcrq', 'pngcwl'], 'fzjdz.mlw': ['/', 'ftj', 'hmd', 'lfpm'], 'glzsnz.cvs': ['/', 'ftj', 'hmd', 'lfpm'], 'sdmmmds.nwt': ['/', 'sbprmc', 'hfns', 'cvjtpc', 'whqnlc', 'tchbjclg', 'bljmjwm', 'qlzg', 'ghllcw', 'vpn'], 'lwhtsvg.bjm': ['/', 'ftj', 'hmd', 'lfpm', 'vlnbm'], 'wtbzpqvb': ['/', 

In [33]:
#stolen

class TreeNode:
    def __init__(self, is_dir: bool, name, size=None) -> None:
        self.is_dir = is_dir
        self.name = name
        self.size = size
        self.children = []
        self.parent = None

    def add_child(self, child):
        child.parent = self
        self.children.append(child)

    def get_size(self):
        if self.is_dir:
            total_size = 0
            for child in self.children:
                total_size += child.get_size()
            return total_size
        else:
            return self.size

    def find_subdirectories_part1(self):
        dir_sizes = 0
        if self.is_dir:
            for child in self.children:
                if child.is_dir and child.get_size() <= 100000:
                    dir_sizes += child.get_size() + child.find_subdirectories_part1()
                else:
                    dir_sizes += child.find_subdirectories_part1()
        return dir_sizes
    
    def find_subdirectories_part2(self, min_size):
        dir_sizes = []
        if self.is_dir:
            for child in self.children:
                if child.is_dir and child.get_size() >= min_size:
                    dir_sizes += [child.get_size()] + child.find_subdirectories_part2(min_size)
                else:
                    dir_sizes += child.find_subdirectories_part2(min_size)
        return dir_sizes

    def print_children(self, level):
        if self.is_dir:
            print('--' * level + self.name + ' (total=' + str(self.get_size()) +')')
        else: 
            print('--' * level + self.name + ' (file=' + str(self.get_size()) +')')
        if len(self.children) > 0:
            for child in self.children:
                child.print_children(level+1)
                
class Tree:
    def __init__(self) -> None:
        self.root = TreeNode(is_dir=True, name="root")
        self.current = self.root 

    def reset_to_root(self):
        self.current = self.root

    def go_up_one_level(self):
        self.current = self.current.parent

    def go_to_child(self, name):
        self.current = list(filter(lambda child: child.name == name, self.current.children))[0]

    def add_new_child(self, child):
        self.current.add_child(child)
        
        
with open('13_feladat.txt', 'r') as f:
    lines = f.readlines()
    lines = [entry.strip() for entry in lines]

tree = Tree()

while len(lines) > 0:
    line = lines.pop(0)
    if line == '$ cd /': 
        tree.reset_to_root()
    elif line == '$ ls':
        while len(lines)>0 and '$' not in lines[0]:
            line = lines.pop(0)
            size, name = line.split(' ')
            if size.isdigit():
                new_node = TreeNode(is_dir=False, name=name, size=int(size))
            else:
                new_node = TreeNode(is_dir=True, name=name)
            tree.add_new_child(new_node)
    elif line == '$ cd ..':
        tree.go_up_one_level()
    elif '$ cd' in line:
        _, _, name = line.split(' ')
        tree.go_to_child(name)


In [34]:

tree.root.print_children(0)
tree.root.find_subdirectories_part1()

root (total=43636666)
--blrnnv (total=169869)
----mjjj.wnq (file=169869)
--ctfjwl (total=209537)
----zzdpcnmt.qbf (file=209537)
--dqf (total=548836)
----hnbqlmmg (file=205200)
----lmw.zmd (file=80316)
----mwj (total=141008)
------bddslhrg.twf (file=18195)
------qfccg.crl (file=122813)
----tsrwvqbg.tzh (file=122312)
--dqqbcfr (file=135993)
--ftj (total=1749433)
----clchr (total=257770)
------sbzf (total=257770)
--------thwhz.pgp (file=257770)
----hmd (total=1451431)
------bddslhrg.qzl (file=62643)
------lfpm (total=985654)
--------bddslhrg (file=107158)
--------fzjdz.mlw (file=253844)
--------glzsnz.cvs (file=217005)
--------qfccg.crl (file=71371)
--------sdmmmds.nwt (file=193163)
--------vlnbm (total=143113)
----------lwhtsvg.bjm (file=143113)
------sgddpc (file=277745)
------vjchlsrc.cmh (file=117478)
------zzdpcnmt.qbf (file=7911)
----vtzlv (file=40232)
--fzjdz (file=125510)
--jvtlfbzr (total=18929272)
----gwz (total=13023723)
------blfnwqtm.vhw (file=161853)
------fbqpql.bbw (file=1

1297159

In [35]:
# 14. feladat:


total_space = 70000000
space_needed = 30000000
current_empty_space = 70000000 - tree.root.get_size()
possible_dirs = tree.root.find_subdirectories_part2(space_needed - current_empty_space)
min(possible_dirs)

3866390